In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#set parameters
make = 'audi'
model = 'a3'
postcode = 'w1h1pg'
radius = '50'
price_to = '10000'
page = 1

#establish number of pages in result set
url = ('http://www.autotrader.co.uk/search/used/cars/%s/%s/postcode/%s/radius/%s/price-to/%s/onesearchad/used/sort/default/page/%s/searchcontext/default')%(make, model, postcode, radius, price_to, page)
x = requests.get(url)
soup = BeautifulSoup(x.text, 'html.parser')

pages = []
for i in soup.find('li', class_ = "paginationMini__count").find_all('strong'):
    pages.append(int(i.string))

In [3]:
#initiate master dataframe
master = []

for q in range(pages[0], pages[1] + 1):
    
    page = q
    url = ('http://www.autotrader.co.uk/search/used/cars/%s/%s/postcode/%s/radius/%s/price-to/%s/onesearchad/used/sort/default/page/%s/searchcontext/default')%(make, model, postcode, radius, price_to, page)
    x = requests.get(url)
    soup = BeautifulSoup(x.text, 'html.parser')
    
    for k in range(0, len(soup.find_all('article', class_ = 'search-result '))):
        ad = soup.find_all('article', class_ = 'search-result ')[k]

        #create list of ad attributes
        attributes = []
        attributes.append(ad.find('h1', class_ = "search-result__title").text)

        #add price to list
        attributes.append(ad.find_all('div', class_= "search-result__price")[0].string)

        #add additional attributes
        for i in ad.find_all('ul', class_="search-result__attributes")[0].find_all('li')[-7:]:
            attributes.append(i.string)

        #add seller details to list
        for i in ad.find_all('p', class_= "search-result__seller-details")[0].find_all('strong'):
            attributes.append(i.string)

        master.append(attributes)
    print 'page ' + str(q)

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
page 25
page 26
page 27
page 28
page 29
page 30
page 31
page 32
page 33
page 34
page 35
page 36
page 37
page 38
page 39
page 40
page 41
page 42
page 43
page 44
page 45
page 46
page 47
page 48
page 49
page 50
page 51
page 52
page 53
page 54
page 55
page 56
page 57
page 58
page 59
page 60
page 61
page 62
page 63
page 64
page 65
page 66
page 67
page 68
page 69
page 70
page 71
page 72
page 73
page 74
page 75
page 76
page 77
page 78
page 79
page 80
page 81
page 82
page 83


In [4]:
df = pd.DataFrame(master)
df.columns = ['model', 'price', 'year', 'type', 'mileage', 'transmission', 'engine_size', 'hp', 'fuel', 'seller_type', 'distance']
df = df.dropna()

In [11]:
df.mileage = pd.to_numeric(df.mileage.str.replace(',| miles', ''))

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [36]:
df.price = df.price.str.replace('£|,', '')

In [24]:
df.year = pd.to_numeric(df.year.str.replace(' \(.*\)', ''))

In [15]:
df.engine_size = pd.to_numeric(df.engine_size.str.replace('L', ''))

In [16]:
df.hp = pd.to_numeric(df.hp.str.replace(' bhp', ''))

In [17]:
df.distance = pd.to_numeric(df.distance.str.replace(' miles', ''))

ValueError: Unable to parse string

In [39]:
df

,model,price,year,type,mileage,transmission,engine_size,hp,fuel,seller_type,distance
0,Audi A3 1.6 SE 3dr,£495,1997,Hatchback,126000,Manual,1.6,101,Petrol,Private seller,22 miles
1,Audi A3 1.8 Sport 3dr,£495,2001,Hatchback,141677,Manual,1.8,125,Petrol,Trade seller,29 miles
2,Audi A3 1.8 SE 5dr,£500,2000,Hatchback,119000,Automatic,1.8,125,Petrol,Trade seller,24 miles
3,Audi A3 1.6 3dr,£550,1999,Hatchback,118000,Manual,1.6,101,Petrol,Private seller,35 miles
4,Audi A3 1.8 T Sport 5dr,£599,2001,Hatchback,147000,Manual,1.8,150,Petrol,Trade seller,9 miles
5,Audi A3 1.6 3dr,£599,2000,Hatchback,157350,Manual,1.6,101,Petrol,Trade seller,9 miles
6,Audi A3 1.8 Sport 3dr,£600,1997,Hatchback,49600,Manual,1.8,125,Petrol,Private seller,8 miles
7,Audi A3 1.6 SE 5dr,£695,2003,Hatchback,106000,Manual,1.6,101,Petrol,Trade seller,45 miles
8,Audi A3 1.8 T Sport Quattro 3dr,£695,2000,Hatchback,150000,Manual,1.8,180,Petrol,Trade seller,41 miles
9,Audi A3 1.6 SE 5dr,£695,2001,Hatchback,130000,Manual,1.6,101,Petrol,Trade seller,6 miles
